In [ ]:
!pip install newspaper3k psycopg2 sentence-transformers faiss-cpu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 67.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install lxml_html_clean

In [ ]:
import faiss
import psycopg2
import numpy as np
from newspaper import Article
from sentence_transformers import SentenceTransformer
import requests
from bs4 import BeautifulSoup


In [ ]:
#POSTGRESQL stuff

import psycopg2

conn = psycopg2.connect(
    dbname="financial_db",
    user="finance_user",
    password="sudhSsudh",\
    host="0.tcp.in.ngrok.io",
    port="14566"
)

cur = conn.cursor()
print("Connected successfully!")

cur.execute("SELECT 1;")
print(cur.fetchone())


Connected successfully!
(1,)


In [ ]:
#FAISS stuff

embedding_dim = 384  # MiniLM uses 384
index = faiss.IndexFlatL2(embedding_dim)

model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
article_urls = [

    "https://www.investopedia.com/terms/l/leveraged-etf.asp",
    "https://www.financialplanningassociation.org/sites/default/files/2021-08/MAY08%20Leveraged%20ETFs%20A%20Risky%20Double%20That%20Doesnt%20Multiply%20by%20Two.pdf",
    "https://www.investopedia.com/articles/exchangetradedfunds/07/leveraged-etf.asp",
    "https://www.investor.gov/introduction-investing/general-resources/news-alerts/alerts-bulletins/investor-alerts/sec",
    "https://www.timeline.co/blog/leveraged-etfs-are-not-worth-the-trouble",
    "https://investor.vanguard.com/investor-resources-education/etfs/leveraged-inverse-etf-etn",
    "https://bankunderground.co.uk/2023/08/31/leveraged-and-inverse-etfs-the-exotic-side-of-exchange-traded-funds/",
    "https://www.britannica.com/money/inverse-leveraged-etf-definition",
    "https://www.cnbc.com/video/2025/02/24/whats-behind-the-explosion-in-leverage-and-inverse-etfs.html",
    "https://www.morningstar.co.uk/uk/news/69408/how-to-use-a-leveraged-or-inverse-etf.aspx",
    "https://www.investopedia.com/articles/mutualfund/07/etf_downside.asp",
    "https://www.ft.com/content/ce896be0-6e33-4636-a648-7af4bf250790",
    "https://www.forbes.com/advisor/investing/best-leveraged-etfs/",
    "https://www.investopedia.com/terms/l/leveraged-etf.asp",
    "https://www.bairdwealth.com/globalassets/pdfs/help/leveraged-inverse-funds.pdf"
    "https://www.thestockdork.com/the-benefits-and-risks-of-trading-options/",
    "https://www.nasdaq.com/articles/the-many-advantages-of-options-trading",
    "https://www.nerdwallet.com/article/investing/options-trading-risks-benefits?utm_source=chatgpt.com",
    "https://www.indiainfoline.com/knowledge-center/derivatives/benefits-of-option-trading",
    "https://www.investopedia.com/buying-vs-selling-options-7972599?utm_source=chatgpt.com",
    "https://www.globalbankingandfinance.com/the-risks-and-rewards-of-trading-options",
    "https://techstory.in/benefits-and-risks-of-options-trading/?utm_source=chatgpt.com",
    "https://intrinio.com/blog/what-are-historical-options-prices-importance-limitations",
    "https://www.indiatoday.in/business/story/sebi-study-fo-traders-7-percent-made-profit-in-last-3-years-93-percent-loss-2605111-2024-09-23?utm_source=chatgpt.com",
    "https://www.business-standard.com/markets/news/over-93-retail-traders-lost-money-in-f-o-in-three-years-says-sebi-124092300853_1.html?utm_source=chatgpt.com",
    "https://www.sahmcapital.com/education/article/big-gains-or-big-risks-the-truth-about-penny-stocks/35?utm_source=chatgpt.com",
    "https://www.finra.org/investors/insights/low-priced-stocks-big-problems?utm_source=chatgpt.com",
    "https://www.fidelity.com/viewpoints/active-investor/trading-penny-stocks?utm_source=chatgpt.com",
    "https://www.nasdaq.com/articles/microcap-stock-definition-how-it-works-pros-cons-strategies?utm_source=chatgpt.com",
    "https://en.wikipedia.org/wiki/Microcap_stock?utm_source=chatgpt.com",
    "https://www.peterleeds.com/blog.htm?blog=423%2F&utm_source=chatgpt.com",
    "https://fastercapital.com/content/Penny-Stocks--Exploring-the-Cult-Status-of-Penny-Stocks.html?utm_source=chatgpt.com",
    "https://pictureperfectportfolios.com/micro-cap-investing-pros-cons/?utm_source=chatgpt.com",
    "https://www.tikr.com/blog/why-small-caps-outperform-and-when-they-dont",
    "https://economictimes.indiatimes.com/mf/analysis/what-are-risks-and-opportunities-associated-with-small-caps/articleshow/113780313.cms",
    "https://www.investopedia.com/terms/s/small-cap.asp",
    "https://www.wellington.com/en/insights/setting-the-stage-for-small-caps-in-2025",
    "https://viewpoint.bnpparibas-am.com/the-outlook-for-us-small-caps-in-2025/",
    "https://www.trustnet.com/news/13434947/what-trends-are-guiding-global-small-caps-in-2025",
    "https://rankiapro.com/en/insights/2025-strong-year-small-mid-cap-stocks/",
    "https://news.umich.edu/cryptocurrencies-high-volatility-and-returns/",
    "https://en.wikipedia.org/wiki/Bankruptcy_of_FTX",
    "https://en.wikipedia.org/wiki/Bitconnect",
    "https://en.wikipedia.org/wiki/2021_Squid_Game_cryptocurrency_scam",
    "https://www.reuters.com/sustainability/bhutan-turns-green-cryptocurrency-fuel-economy-2025-04-17/",
    "https://www.businessinsider.com/retail-investors-trump-trade-war-stock-market-panic-bitcoin-tariffs-2025-4#:~:text=Leveraged%20ETFs%20use%20debt%20to,returns%20but%20also%20amplified%20losses.",
    "https://jfin-swufe.springeropen.com/articles/10.1186/s40854-024-00646-y",
    "https://www.mdpi.com/1911-8074/16/2/136",
    "https://disb.dc.gov/page/you-invest-crypto-know-risks#:~:text=It%20could%20also%20be%20a,you%20can%20afford%20to%20lose.",
    "https://corporatefinanceinstitute.com/resources/cryptocurrency/is-cryptocurrency-a-good-investment/#:~:text=Summary,requirements%20fit%20their%20investor%20profile.",
    "https://hbr.org/2022/07/what-skeptics-get-wrong-about-cryptos-volatility",
    "https://www.fidelitydigitalassets.com/research-and-insights/closer-look-bitcoins-volatility",
    "https://economictimes.indiatimes.com/wealth/invest/is-cryptocurrency-story-over-4-things-crypto-investors-should-know-to-navigate-the-high-risk-arena/articleshow/92441791.cms?from=mdr",
    "https://www.sciencedirect.com/science/article/abs/pii/S1057521923002284",
    "https://www.sciencedirect.com/science/article/pii/S1044028321000399",
    "https://economictimes.indiatimes.com/markets/cryptocurrency/navigating-uncertainty-insights-on-trading-in-a-volatile-crypto-market/articleshow/112372794.cms?from=mdr",
    "https://portal.ct.gov/dob/consumer/consumer-education/cryptocurrency-risks#:~:text=Cryptocurrencies%20aren't%20backed%20by,protections%20as%20a%20bank%20account.",
    "https://kriptomat.io/finance-investing/what-are-the-risks-and-rewards-of-buying-newly-launched-cryptocurrencies/",
    "https://www.cnbc.com/2025/01/11/bitcoin-soared-in-2024-how-much-if-any-should-you-own.html#:~:text=Bitcoin%20was%20the%20top%2Dperforming,a%20place%20in%20investment%20portfolios.",
    "https://www.fool.com/investing/2025/02/26/from-volatility-to-fraud-is-investing-in-cryptocur/#:~:text=From%20Volatility%20to%20Fraud%2C%20Is%20Investing%20in%20Cryptocurrency%20Just%20Too%20Risky%3F,-By%20James%20Brumley&text=The%20rally%20in%20Bitcoin%20and,simple%20fear%20of%20missing%20out.",
    "https://calebandbrown.com/blog/crypto-volatility/",
    "https://www.brookings.edu/articles/the-brutal-truth-about-bitcoin/",
    "https://www.nasdaq.com/articles/2-ways-people-are-making-millions-meme-coins-and-why-its-dangerous-investment",
    "https://www.bitpanda.com/academy/en/lessons/ethereum-forecast-2025-trends-scenarios-and-expert-opinions/",
    "https://tfoco.com/en/insights/articles/rise-and-fall-of-terra",
    "https://www.cointree.com/learn/bitcoin-success-stories/",
    "https://in.investing.com/news/expert-analyst-famous-for-predicting-solanas-comeback-from-2022-lows-says-this-token-is-about-to-go-vertical-with-up-to-2500-returns-expected-in-the-next-3-months-4122833",
    "https://www.nerdwallet.com/article/investing/meme-coins",

    "https://www.investopedia.com/terms/b/bond.asp",
    "https://www.investopedia.com/financial-edge/0312/the-basics-of-bonds.aspx",
    "https://www.investopedia.com/ask/answers/05/bondrisks.asp",
    "https://www.treasurydirect.gov/marketable-securities/",
    "https://www.investopedia.com/articles/economics/09/why-gold-matters.asp",
    "https://www.investopedia.com/financial-edge/0311/what-drives-the-price-of-gold.aspx",
    "https://www.investopedia.com/terms/b/bullion.asp",  # multi
    "https://www.kitco.com/opinion/2025-03-31/golds-stellar-performance-largest-monthly-percentage-gain-february-2016",
    "https://www.investopedia.com/terms/s/silver.asp",
    "https://www.investopedia.com/silver-price-rising-more-than-gold-in-april-8629286",
    "https://www.investopedia.com/how-to-invest-in-gold-and-silver-7369625",# multi
    "https://www.moneycontrol.com/news/silver-rates-today/",

]

#

In [ ]:
conn.rollback()

In [ ]:
#Simulate user and get etxt from articles

def extract_article_bs(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")

        title = soup.find("title").text if soup.find("title") else "No Title Found"

        paragraphs = soup.find_all("p")
        content = "\n".join([p.text for p in paragraphs])

        if len(content) < 100:
            raise ValueError("Extracted content is too short")

        return title, content

    except Exception as e:
        print(f"Error extracting {url}: {e}")
        return None, None

In [ ]:
#Stroing in FAISS and postgresql
for url in article_urls:
    try:
        title, text = extract_article_bs(url)
        if not text:
            continue
        '''
        # Insertiosn into postgresql(TABLE = news_articles )
        cur.execute("INSERT INTO risky_assets(title, url, content) VALUES (%s, %s, %s) RETURNING id;",
                    (title, url, text))
        article_id = cur.fetchone()[0]
        conn.commit()
        print(f"Inserted: {title}")
        '''
        embedding = model.encode(text, convert_to_numpy=True)

        # Storing in FAISS
        index.add(np.array([embedding], dtype=np.float32))

        print(f"Stored: {title}")

    except psycopg2.Error as db_err:
        conn.rollback()
        print(f"Database error: {db_err}")

    except Exception as e:
        print(f"Error processing {url}: {e}")

#This save sthe created FAISS index
faiss.write_index(index, "news_faiss.index")

Stored: Leveraged ETFs: The Potential for Big Gains—and Bigger Losses
Error extracting https://www.financialplanningassociation.org/sites/default/files/2021-08/MAY08%20Leveraged%20ETFs%20A%20Risky%20Double%20That%20Doesnt%20Multiply%20by%20Two.pdf: Extracted content is too short
Stored: Dissecting Leveraged ETF Returns: Explanation, Components, and Examples
Stored: Updated Investor Bulletin: Leveraged and Inverse ETFs | Investor.gov
Stored: Leveraged ETFs Are Not Worth the Trouble
Stored: Leveraged, inverse, and commodity ETFs and ETNs | Vanguard
Stored: Leveraged and inverse ETFs – the exotic side of exchange-traded funds – Bank Underground
Stored: Inverse and Leveraged ETFs: How Do They Work? | Britannica Money
Stored: What's behind the explosion in leverage and inverse ETFs
Stored: 
	How to Use a Leveraged or Inverse ETF | Morningstar

Stored: 10 ETF Concerns That Investors Shouldn’t Overlook
Stored: The dangers of leveraged ETFs 
Stored: 8 Best Leveraged ETFs of 2025 – Forbes Advis

In [ ]:
conn.commit()


In [ ]:
import faiss
import numpy as np

# Loading the Faiss saved index
index = faiss.read_index("news_faiss.index")

print(f"Number of embeddings in FAISS: {index.ntotal}")


Number of embeddings in FAISS: 70


In [ ]:
conn.rollback()

In [ ]:
print(index.ntotal)
cur.execute("SELECT COUNT(*) FROM risky_assets");
print(cur.fetchone())


70
(69,)


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


#Get similar embeddings to query and then bget content from postgresql
def retrieve_articles(query, top_k=5):
    query_embedding = embedding_model.encode(query, convert_to_numpy=True)
    print("Query embedding (first 5):", query_embedding[:5])
    D, I = index.search(np.array([query_embedding], dtype=np.float32), 5)
    print("Distances:", D)
    print("Indices:", I)

    _, indices = index.search(np.array([query_embedding], dtype=np.float32), top_k)

    cur.execute("SELECT title, content FROM risky_assets WHERE id = ANY(%s);", (list(map(int, indices[0])),))

    return cur.fetchall()

query = "Talk about smallcap and microcap stocks"
retrieved_articles = retrieve_articles(query)
for title, content in retrieved_articles:
    print(f"🔹 {title}\n{content[:300]}...\n")


Query embedding (first 5): [0.0706484  0.00507676 0.03712593 0.03437687 0.08909021]
Distances: [[0.47689503 0.4819008  0.54336965 0.6755673  0.76518387]]
Indices: [[25 24 28 31 34]]


In [ ]:
# Get the number of articles in the database

id_map = {}
cur.execute("SELECT COUNT(*) FROM risky_assets;")
num_articles = cur.fetchone()[0]

# Loop only through the minimum of FAISS index count and DB rows
for faiss_index in range(min(index.ntotal, num_articles)):
    cur.execute("SELECT id FROM risky_assets LIMIT 1 OFFSET %s;", (faiss_index,))
    result = cur.fetchone()

    if result:
        article_id = result[0]
        id_map[faiss_index] = article_id

        cur.execute("""
            INSERT INTO faiss_postgres (faiss_index, article_id)
            VALUES (%s, %s)
            ON CONFLICT (faiss_index) DO NOTHING;
        """, (faiss_index, article_id))
        conn.commit()


In [ ]:
import pickle
with open("article_id_map.pkl", "wb") as f:
    pickle.dump(id_map, f)

In [ ]:
# retrieving articels from FAISS search

query="gold and silver"
query_embedding = embedding_model.encode(query, convert_to_numpy=True)
D, I = index.search(np.array([query_embedding], dtype=np.float32), 5)

# Loadign the map
with open("article_id_map.pkl", "rb") as f:
    id_map = pickle.load(f)

matched_ids = [id_map[i] for i in I[0]]

cur.execute("SELECT title, content FROM risky_assets WHERE id = ANY(%s);", (matched_ids,))
retrieved_articles = cur.fetchall()

for title, content in retrieved_articles:
    print(f"🔹 {title}\n{content[:300]}...\n")


🔹 Understanding the Dynamics Behind Gold Prices
An In-Depth Look at the Economic and Market Influences on Gold
Adam Hayes, Ph.D., CFA, is a financial writer with 15+ years Wall Street experience as a derivatives trader. Besides his extensive derivative trading expertise, Adam is an expert in economics and behavioral finance. Adam received his mas...

🔹 Silver: What It Is, How It Works, and History
Gordon Scott has been an active investor and technical analyst or 20+ years. He is a Chartered Market Technician (CMT).
m.czosnek / Getty Images
 The term “silver” refers to a precious metal commonly used in the production of jewelry, coins, electronics, and photography. It has the highest electrica...

🔹 Why Silver is Outshining Gold in April
Key Chart Levels to Monitor As The Grey Metal Has Gained 12% This Month
 Silver (SILVER) price gains may be somewhat dull compared gold’s (GOLD) glistening returns over the past year. But since the start of this month, the gray metal has added 12% compa

In [ ]:
import os
#os.environ["OPENAI_API_KEY"] = #openai key


In [ ]:
!pip install openai

In [ ]:
import openai

query="Leveraged etfs and smallcap stocks"
query_embedding = embedding_model.encode(query, convert_to_numpy=True)
D, I = index.search(np.array([query_embedding], dtype=np.float32), 5)

with open("article_id_map.pkl", "rb") as f:
    id_map = pickle.load(f)

matched_ids = [id_map[i] for i in I[0]]

cur.execute("SELECT title, content FROM risky_assets WHERE id = ANY(%s);", (matched_ids,))
retrieved_articles = cur.fetchall()

for title, content in retrieved_articles:
    print(f"🔹 {title}\n{content[:300]}...\n")


#openai.api_key = "KEY"

def generate_response(query, retrieved_articles):
    context = "\n\n".join([f"Title: {title}\nContent: {content[:1000]}" for title, content in retrieved_articles])
    prompt = f"Using the following news articles, answer this query: {query}\n\n{context}\n\nResponse:"

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a financial analyst."},
                  {"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content


cot_prompt = """
User: Comapre the performance of leveraged etfs to that of small and micro cap stocks
Assistant: Let's think step by step.
"""

retrieved_articles = retrieve_articles(cot_prompt)
response = generate_response(query, retrieved_articles)
print(response)


🔹 Dissecting Leveraged ETF Returns: Explanation, Components, and Examples
 Most exchange-traded funds (ETFs) seek to duplicate the returns of an index or other benchmark. Leveraged exchange-traded funds (ETFs) try to deliver a greater return.

 As with other ETFs, the investor can choose an ETF that is based on the S&P 500 Index or the Nasdaq 100, or one of many that trac...

🔹 Leveraged ETFs Are Not Worth the Trouble
Leveraged exchange traded fund ETFs are apparently selling well. Morningstar estimates that, in the US alone, they saw net inflows of around $13 million in the 12 months to the end of January 2025. 
These products are now firmly established. For example, the world’s largest leveraged ETF, ProShares ...

🔹 
	How to Use a Leveraged or Inverse ETF | Morningstar

Get 14 Days Free

Jason Stipp: I am Jason Stipp for Morningstar.
It's ETF Investing Week on Morningstar.com, and today we are answering your ETF questions.
I'm talking with Paul Justice; he is director of North Ameri

In [ ]:
import openai


query="Leveraged etfs and options and cryptocurrency"
query_embedding = embedding_model.encode(query, convert_to_numpy=True)
D, I = index.search(np.array([query_embedding], dtype=np.float32), 5)

with open("article_id_map.pkl", "rb") as f:
    id_map = pickle.load(f)

matched_ids = [id_map[i] for i in I[0]]

cur.execute("SELECT title, content FROM risky_assets WHERE id = ANY(%s);", (matched_ids,))
retrieved_articles = cur.fetchall()


for title, content in retrieved_articles:
    print(f"🔹 {title}\n{content[:300]}...\n")


#openai.api_key = "KEY"

def generate_response(query, retrieved_articles):
    context = "\n\n".join([f"Title: {title}\nContent: {content[:1000]}" for title, content in retrieved_articles])
    prompt = f"Using the following news articles, answer this query: {query}\n\n{context}\n\nResponse:"

    response = openai.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{"role": "system", "content": "You are a financial analyst."},
                  {"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

tot_prompt = """
User: I want to diversify my portfolio with high-risk assets.
Assistant: Let's explore multiple options based on different preferences.

Option 1: Cryptocurrency
Option 2: Options
Option 3: Leveraged etfs
Other Other assets based on articles

Based on user goals (comfortable with very risky assets), let's reason through each and pick the best.

Format the output well and give a nice response to the user
"""

retrieved_articles = retrieve_articles(tot_prompt)
response = generate_response(tot_prompt, retrieved_articles)
print(response)



🔹 Leveraged ETFs: The Potential for Big Gains—and Bigger Losses
Gordon Scott has been an active investor and technical analyst or 20+ years. He is a Chartered Market Technician (CMT).
A leveraged exchange-traded fund (LETF) uses financial derivatives and debt to amplify the returns of an underlying index, stock, specific bonds, or currencies.
 A leveraged exchan...

🔹 Dissecting Leveraged ETF Returns: Explanation, Components, and Examples
 Most exchange-traded funds (ETFs) seek to duplicate the returns of an index or other benchmark. Leveraged exchange-traded funds (ETFs) try to deliver a greater return.

 As with other ETFs, the investor can choose an ETF that is based on the S&P 500 Index or the Nasdaq 100, or one of many that trac...

🔹 
	How to Use a Leveraged or Inverse ETF | Morningstar

Get 14 Days Free

Jason Stipp: I am Jason Stipp for Morningstar.
It's ETF Investing Week on Morningstar.com, and today we are answering your ETF questions.
I'm talking with Paul Justice; he is di

In [ ]:
!pip install langchain openai

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23


In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.3 MB/s eta 0:00:00


In [ ]:
# retrieving articels from FAISS search
'''
query="gold and silver"
query_embedding = embedding_model.encode(query, convert_to_numpy=True)
D, I = index.search(np.array([query_embedding], dtype=np.float32), 5)

# Loadign the map
with open("article_id_map.pkl", "rb") as f:
    id_map = pickle.load(f)

matched_ids = [id_map[i] for i in I[0]]

cur.execute("SELECT title, content FROM risky_assets WHERE id = ANY(%s);", (matched_ids,))
retrieved_articles = cur.fetchall()

for title, content in retrieved_articles:
    print(f"🔹 {title}\n{content[:300]}...\n")
'''


#openai.api_key = "KEY"

def retrieve_articles(query, top_k=5):

    query_embedding = embedding_model.encode(query, convert_to_numpy=True)

    D, I = index.search(np.array([query_embedding], dtype=np.float32), 5)

    # Loadinf the FAISS-POSTGRESQL MAP
    with open("article_id_map.pkl", "rb") as f:
        id_map = pickle.load(f)

    # Convertign FAISS indices to actual DB IDs
    matched_ids = [id_map[i] for i in I[0]]

    # Retrieve from PostgreSQL
    cur.execute("SELECT title, content FROM risky_assets WHERE id = ANY(%s);", (matched_ids,))
    articles = cur.fetchall()

    return articles


def generate_response(query, retrieved_articles):
    context = "\n\n".join([f"Title: {title}\nContent: {content[:1000]}" for title, content in retrieved_articles])
    prompt = f"Using the following news articles, answer this query: {query}\n\n{context}\n\nResponse:"

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a financial analyst."},
                  {"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

tot_prompt = """
User: I want to diversify my portfolio
Assistant: Let's explore multiple assets based on different preferences.

1: Treasury bonds — low volatility, fixed returns.
2: Gold — good inflation hedge, moderate liquidity.
3: Silver — value store + industrial demand, higher volatility.


Based on user goals let's reason through each and pick the best.
"""

retrieved_articles = retrieve_articles(tot_prompt)
response = generate_response(tot_prompt, retrieved_articles)
print(response)


Based on your goal to diversify your portfolio, here are some considerations for each asset:

1. Treasury Bonds: Treasury bonds are known for their low volatility and fixed returns, making them a relatively stable investment option. They are backed by the full faith and credit of the United States government, providing a high level of safety. If you are looking for a low-risk asset with predictable returns, treasury bonds could be a suitable choice.

2. Gold: Gold serves as a good hedge against inflation and is often seen as a store of value during economic uncertainties. It also offers moderate liquidity, allowing you to easily buy and sell gold when needed. However, gold prices can be volatile in the short term, so it may not be suitable for those seeking stable returns.

3. Silver: Silver combines elements of a value store and industrial demand, offering potential for both investment and practical use. It tends to have higher volatility compared to gold, which can present both oppor

In [ ]:
def retrieve_articles(query, top_k=5):
    query_embedding = embedding_model.encode(query, convert_to_numpy=True)

    D, I = index.search(np.array([query_embedding], dtype=np.float32), top_k)

    with open("article_id_map.pkl", "rb") as f:
        id_map = pickle.load(f)

    matched_ids = [id_map[i] for i in I[0]]

    cur.execute("SELECT title, content FROM risky_assets WHERE id = ANY(%s);", (matched_ids,))
    articles = cur.fetchall()

    return articles


#openai.api_key = "KEY"

def generate_response(query, top_k=5):
    articles = retrieve_articles(query, 5)

    if not articles:
        return "No relevant articles found."

    context = "\n\n".join([
        f"Title: {title}\nContent: {content[:1000]}"
        for title, content in articles
    ])

    prompt = f"Using the following news articles, answer this query: {query}\n\n{context}\n\nResponse:"

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a financial analyst. Use the provided context and the conversation history to answer the user's question. If the user provides personal information (e.g., their name), remember it and use it in future responses when relevant"},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content


In [ ]:
from langchain.tools import Tool


def article_tool_func(query: str):
    return retrieve_articles(query)

def gpt_tool_func(query: str):
    articles = retrieve_articles(query)
    return generate_response(query, articles)

tools = [
    Tool(name="retrieve_articles", func=article_tool_func, description="Retrieve financial articles based on a query."),
    Tool(name="generate_response", func=gpt_tool_func, description="Generate GPT response from financial articles.")
]


In [ ]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory



llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    memory=memory,
    verbose=True
)


<ipython-input-38-43c7751b1005>:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
<ipython-input-38-43c7751b1005>:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
<ipython-input-38-43c7751b1005>:15: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For detai

In [ ]:
query = "What are the risks associated with leveraged ETFs?"
response = agent.run(query)

print(response)



> Entering new AgentExecutor chain...

Invoking: `retrieve_articles` with `risks of leveraged ETFs`


[('Dissecting Leveraged ETF Returns: Explanation, Components, and Examples', ' Most exchange-traded funds (ETFs) seek to duplicate the returns of an index or other benchmark. Leveraged exchange-traded funds (ETFs) try to deliver a greater return.\n\n As with other ETFs, the investor can choose an ETF that is based on the S&P 500 Index or the Nasdaq 100, or one of many that tracks a specific sector or industry group.\n\n Like stocks, shares of ETFs are traded on a public exchange. Like mutual funds, ETFs allow individual investors to benefit from the diversification and economies of scale that come from joining a large pool of money contributed by many investors.\n\n Most ETFs are passive investments. They invest strictly in the stocks, bonds, or other assets that are listed in a particular index such as the S&P 500, buying and selling only when the index is revised.\n\n The first lev

In [ ]:
print("Agent Chatbot with Memory (type 'exit' to quit)")

while True:

    user_input = input("Make use of the retriever tool to get atleast 5 relevant articles related to the query")
    if user_input.lower() in ['exit', 'quit']:
        break

    response = agent.run(user_input)
    print("Agent:", response)


Agent Chatbot with Memory (type 'exit' to quit)
Make use of the retriever tool to get atleast 5 relevant articles related to the queryhi my name is sudhanva


> Entering new AgentExecutor chain...
Hello Sudhanva! How can I assist you today?

> Finished chain.
Agent: Hello Sudhanva! How can I assist you today?
Make use of the retriever tool to get atleast 5 relevant articles related to the querycan you talk about some risky assets to invest in?


> Entering new AgentExecutor chain...

Invoking: `retrieve_articles` with `risky assets to invest in`


[('Benefits and Risks of Options Trading – TechStory', "Photo by Jason Briscoe on Unsplash\nOptions trading involves a derivatives contract that gives the right, but not the obligation, to buy or sell an asset at a predetermined price before a set date.\nThis trading tool is getting popular among traders because of its high return potential, numerous possibilities as well as protection against fluctuations in the market. However, it also carr

KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install gradio

In [ ]:
def answer(user_input):

    return agent.run(user_input)



In [ ]:
# def answer_question(query):
#     return retrieval_chain.invoke(query)
import gradio as gr

chatbot_ui = gr.Interface(
fn=answer,
inputs=gr.Textbox(lines=2, placeholder="Ask me any assets, safe or risky "),
outputs="text",
title="Invesment Advisor Chatbot",
description="Ask questions about market trends, past performance, risks",
)
chatbot_ui.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a961f2882175f3bbf5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
Hello Sudhanva! How can I assist you today?

> Finished chain.


> Entering new AgentExecutor chain...
Sure, investing in risky assets can potentially lead to higher returns, but it also comes with higher risks. Before you proceed, it's important to do thorough research and understand the potential risks involved. 

Would you like me to retrieve some financial articles on investing in risky assets to help you make an informed decision?

> Finished chain.


> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a961f2882175f3bbf5.gradio.live
